In [1]:
import re
from geoip2.database import Reader

In [2]:
sc

<SparkContext master=spark://devenv:7077 appName=PySparkShell>

In [3]:
def log_pattern():
    three_digits = "\\d{1,3}"
    ip = "({}\\.{}\\.{}\\.{})?".format(three_digits, three_digits, three_digits, three_digits)
    client = "(\\S+)"
    uid = "(\\S+)"
    date_time = "(\\[.+?\\])"
    request = "\"(.*?)\""
    status_code = "(\\d{3})"
    bytes_part = "(\\S+)"
    referer = "\"(.*?)\""
    agent = "\"(.*?)\""
    regex = "{} {} {} {} {} {} {} {} {}".format(ip, client, uid, date_time, request, status_code, bytes_part, referer, agent)
    return re.compile(regex)

In [4]:
pattern = log_pattern()

In [5]:
reader = None

In [6]:
lines = sc.textFile("hdfs://devenv/user/spark/spark101/access_log_analysis/data")

In [7]:
def extract_location(line):
    global reader

    if reader is None:
        reader = Reader("/home/spark/spark-2.4.5-bin-hadoop2.7/maxmind/GeoLite2-City.mmdb")

    match = pattern.match(line)

    if match:
        ip = match.group(1)
        response = reader.city(ip)
        country = response.country.name
        city = response.city.name

        if city is None:
            return country
        else:
            return "{},{}".format(country, city)

    else:
        return "InvalidLogFound"

In [8]:
ips = lines.map(extract_location)

In [9]:
location_visits = ips.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

In [10]:
location_visits_sorted = location_visits.sortBy(lambda x: x[1], False)

In [11]:
result = location_visits_sorted.collect()

In [12]:
for ip, count in result:
    print("{}: {}".format(ip, count))

Netherlands: 68493
United States,Ashburn: 4434
France: 1756
United States: 326
China,Beijing: 300
United States,Mountain View: 255
France,Noisy-le-Grand: 81
United States,Redmond: 38
United States,Newburyport: 29
United States,Jacksonville: 27
United Kingdom: 25
United States,Provo: 23
Germany: 23
Russia,St Petersburg: 19
Canada,Montreal: 18
United States,Orlando: 17
United States,San Jose: 11
United States,Chicago: 10
United States,Los Angeles: 10
United States,Fort Lauderdale: 10
Argentina,Buenos Aires: 9
United States,Boydton: 7
United Kingdom,Slough: 6
United States,Sunnyvale: 6
Netherlands,Amsterdam: 5
United States,Spring: 5
Seychelles: 5
United States,Ann Arbor: 5
United States,Clifton: 5
United States,New York: 4
China,Chongqing: 4
Germany,Munich: 4
China,Shenzhen: 4
United States,Houston: 4
United States,Seattle: 4
China,Changsha: 4
Czechia: 4
United States,Englewood: 3
United States,Kansas City: 3
United States,Scottsdale: 3
Ukraine: 3
United States,Fremont: 3
United Kingdom,